In [7]:
"""多层感知机"""
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0943,  0.2088,  0.0693,  0.0480, -0.1652,  0.0269,  0.1009, -0.0070,
         -0.1562, -0.0351],
        [-0.0926,  0.1489,  0.1189,  0.1877, -0.0198, -0.0379,  0.0344, -0.1002,
         -0.1274,  0.0230]], grad_fn=<AddmmBackward0>)

In [5]:
"""自定义块"""
class MLP(nn.Module):
  def __init__(self):
   super().__init__()
   self.hidden = nn.Linear(20, 256)
   self.out = nn.Linear(256, 10)

  def forward(self, X):
   return self.out(F.relu(self.hidden(X))) 

In [8]:
"""实例化多层感知机的层，然后再每次调用正向传播函数时调用这些层"""
net = MLP()
net(X) ## python语法糖，net实际上调用了net.__call__()方法，而__call__()调用了forward（）

tensor([[-0.1041, -0.0833, -0.0232,  0.0491, -0.3226, -0.1230,  0.1440,  0.0560,
         -0.1342, -0.0602],
        [-0.0658, -0.1549, -0.0260,  0.0518, -0.2402, -0.1128,  0.0375,  0.0902,
         -0.1117, -0.0742]], grad_fn=<AddmmBackward0>)

In [9]:
"""顺序块"""
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for block in args:
      self._modules[block] = block

  def forward(self, X):
    for block in self._modules.values():
      X = block(X)
    return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0286, -0.0107,  0.0640,  0.0506,  0.1969, -0.0147,  0.1441,  0.0127,
         -0.1519,  0.1404],
        [-0.0264, -0.1142,  0.0057,  0.0882,  0.1192, -0.1239, -0.0300,  0.0339,
         -0.1740,  0.1343]], grad_fn=<AddmmBackward0>)

In [10]:
"""在正向传播函数中执行代码"""
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.rand_weight = torch.rand((20,20), requires_grad=False)
    self.linear = nn.Linear(20, 20)

  def forward(self, X):
    X = self.linear(X)
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    X = self.linear(X)
    while X.abs().sum() > 1:
      X /= 2
    return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0749, grad_fn=<SumBackward0>)

In [12]:
"""混合搭配各种组合的方法"""
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                  nn.Linear(64, 32), nn.ReLU())
    self.linear = nn.Linear(32, 16)

  def forward(self, X):
    return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1777, grad_fn=<SumBackward0>)